In [1]:
import pandas as pd
import os
import numpy as np
import ReadZTRAX as rz
df = pd.read_csv('FloridaZipData.csv') # https://worldpopulationreview.com/zips/florida

In [3]:
path2zillow = '/Users/taozang/Zillow' # global path to Zillow data
state = '12' # florida
zip_list = df[df['county'].isin(['Miami-Dade'])]['zip'].to_numpy() # miami area zipcode
# change the dtype to 'string'
zip_list = zip_list.astype('str')
zip_list

array(['33012', '33186', '33157', '33015', '33125', '33142', '33033',
       '33177', '33161', '33165', '33018', '33175', '33196', '33126',
       '33176', '33178', '33016', '33032', '33193', '33147', '33162',
       '33155', '33010', '33179', '33172', '33014', '33169', '33055',
       '33134', '33141', '33160', '33139', '33135', '33183', '33056',
       '33133', '33030', '33174', '33156', '33173', '33180', '33127',
       '33054', '33013', '33185', '33143', '33145', '33138', '33130',
       '33150', '33144', '33168', '33189', '33166', '33137', '33034',
       '33184', '33167', '33181', '33140', '33187', '33131', '33182',
       '33146', '33035', '33190', '33136', '33154', '33170', '33132',
       '33129', '33149', '33194', '33128', '33031', '33158', '33109',
       '33039', '33122', '33101'], dtype='<U21')

In [10]:
# PULL ZASMT DATA    
# Load ZAsmt Main spreadsheet (which contains property ZIP) for parcels in interesting ZIP codes
Main = rz.read_ZAsmt_wide(stcate_code=state, 
                          table_name='Main', 
                          row_crit_field='PropertyZip', 
                          row_crit_content=zip_list, 
                          path2zillow = path2zillow)
# specific the directory to store extracted zillow asmt data
os.chdir('/Users/taozang/Zillow/Extracted_Data_MIAMI/')
if not os.path.exists('{}_ZAsmt_CSV'.format(state)):  # check if directory exists
    os.mkdir('{}_ZAsmt_CSV'.format(state))
Main.to_csv('{}_ZAsmt_CSV/Main.csv'.format(state), index=False)
Main.head()

RowID ImportParcelID   FIPS State  \
2138972  B0D8AD01-0000-0000-2EF7-000000000000       28170416  12023    FL   
5351397  9B77DC01-0000-0000-2F36-000000000000       31225755  12086    FL   
5351398  9C77DC01-0000-0000-2F36-000000000000       31225756  12086    FL   
5351399  9D77DC01-0000-0000-2F36-000000000000       31225757  12086    FL   
5351400  9E77DC01-0000-0000-2F36-000000000000       31225758  12086    FL   

             County ValueCertDate ExtractDate Edition ZVendorStndCode  \
2138972    COLUMBIA    2020-07-01      012020      28             SII   
5351397  Miami Dade    2020-07-01      102020      28             SII   
5351398  Miami Dade    2020-07-01      102020      28             SII   
5351399  Miami Dade    2020-07-01      102020      28             SII   
5351400  Miami Dade    2020-07-01      102020      28             SII   

        AssessorParcelNumber  ... PropertyAddressCBSACode  \
2138972   16-5S-17-09272-004  ...                     NaN   
5351397     01-0100-000-0020  ...                     NaN   
5351398     01-0100-000-0022  ...                     NaN   
5351399     01-0100-000-0026  ...                     NaN   
5351400     01-0100-000-0030  ...                     NaN   

        PropertyAddressCBSADivisionCode  PropertyAddressMatchType  \
2138972                             NaN                         1   
5351397                             NaN                         1   
5351398                             NaN                         1   
5351399                             NaN                         1   
5351400                             NaN                         1   

        PropertyAddressDPV PropertyGeocodeQualityCode  \
2138972                                          Zip5   
5351397                                          Zip5   
5351398                                        Parcel   
5351399                  D                     Parcel   
5351400                                        Parcel   

        PropertyAddressQualityCode SubEdition     BatchID BKFSPID SourceChkSum  
2138972                    VeryLow          0  1001750573      -1          NaN  
5351397                    VeryLow          0  1001559308      -1          NaN  
5351398                       High          0  1001559308      -1          NaN  
5351399                       High          0  1001559308      -1          NaN  
5351400                       High          0  1001559308      -1          NaN  

[5 rows x 95 columns]

In [13]:
# Extract list of uniq IDs for parcels from Main
uniq_parcels = Main['RowID'].drop_duplicates().tolist()
tables = ['AdditionalPropertyAddress', 'Building', 'BuildingAreas', 'CareOfName', 
          'ExteriorWall', 'ExtraFeature', 'Garage', 'InteriorFlooring', 'InteriorWall', 
          'LotSiteAppeal', 'MailAddress', 'Main', 'Name', 'Oby', 'Pool', 'SaleData', 'TaxDistrict', 
          'TaxExemption', 'TypeConstruction', 'Value', 'VestingCodes']

rz.tic()
for table in tables:
    foo = rz.read_ZAsmt_wide(state_code=state, 
                             table_name='{}'.format(table), 
                             row_crit_field='RowID', 
                             row_crit_content=uniq_parcels,
                             path2zillow=path2zillow)
    foo.to_csv('{}_ZAsmt_CSV/{}.csv'.format(state, table), index=False)
    print(table)
print('Data extraction done!')
rz.toc()

AdditionalPropertyAddress
Building
BuildingAreas
CareOfName
ExteriorWall
ExtraFeature
Garage
InteriorFlooring
InteriorWall
LotSiteAppeal
MailAddress
Main
Name
Oby
Pool
SaleData
TaxDistrict
TaxExemption
TypeConstruction
Value
VestingCodes
Data extraction done!
Elapsed time is 498.40289402008057 seconds, or 8.306722847620646 minutes.
